In [1]:
!pip install -q lightgbm


In [2]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
import glob
import boto3
import itertools


In [3]:
# Load environment variables
load_dotenv()

AWS_S3_BUCKET = os.getenv("AWS_S3_BUCKET")
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")


In [4]:
# 1. Import & clean
fichiers = sorted(glob.glob('../data/dvf20*.csv'))
dfs = []
for f in fichiers:
    dftmp = pd.read_csv(f, dtype=str)
    dftmp.columns = [col.strip().replace(" ", "_").replace("-", "_").replace("é", "e").replace("É", "E").lower() for col in dftmp.columns]
    if 'insee_com' in dftmp.columns:
        dftmp = dftmp.rename(columns={'insee_com': 'code_commune_insee'})
    if 'codepostal' in dftmp.columns:
        dftmp = dftmp.rename(columns={'codepostal': 'code_postal'})
    dfs.append(dftmp)

df = pd.concat(dfs, ignore_index=True)
df = df.loc[:, ~df.columns.duplicated()]
for col in ['code_commune_insee', 'code_postal']:
    if col in df.columns:
        df[col] = df[col].astype(str).str.zfill(5)
df['annee'] = pd.to_numeric(df['annee'], errors='coerce').fillna(0).astype(int)
for c in ['nb_mutations', 'nbmaisons', 'nbapparts', 'propmaison', 'propappart', 'prixmoyen', 'prixm2moyen', 'surfacemoy']:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0)

colonnes_utiles = [
    'code_commune_insee', 'annee', 'nb_mutations', 'nbmaisons', 'nbapparts',
    'propmaison', 'propappart', 'prixmoyen', 'prixm2moyen', 'surfacemoy'
]
df = df[[col for col in colonnes_utiles if col in df.columns]].drop_duplicates()

# Export prix clean
df.to_csv('df_prix_clean.csv', index=False, encoding='utf-8')
print("Export : df_prix_clean.csv")


Export : df_prix_clean.csv


In [6]:
# 2. Fusion coordonnées
df_coord = pd.read_csv('../data/ref_espace_communes.csv', dtype=str)
df_coord.columns = [col.strip().replace(" ", "_").replace("-", "_").replace("é", "e").replace("É", "E").lower() for col in df_coord.columns]
col_insee_coord = [col for col in df_coord.columns if "insee" in col][0]
df_coord = df_coord.rename(columns={col_insee_coord: 'code_commune_insee'})
df_coord['code_commune_insee'] = df_coord['code_commune_insee'].astype(str).str.zfill(5)
df_coord = df_coord.drop_duplicates(subset=['code_commune_insee'])
# add paris with insee code 75056
df_coord = pd.concat([df_coord, pd.DataFrame({'code_commune_insee': '75056', 'latitude': '48.8566', 'longitude': '2.3522'}, index=[0])], ignore_index=True)


df_merged = pd.merge(df, df_coord[['code_commune_insee', 'latitude', 'longitude']], on='code_commune_insee', how='left')
df_merged['latitude'] = pd.to_numeric(df_merged['latitude'], errors='coerce')
df_merged['longitude'] = pd.to_numeric(df_merged['longitude'], errors='coerce')

# Export merged clean
df_merged.to_csv('df_merged_clean.csv', index=False, encoding='utf-8')
print("Export : df_merged_clean.csv")


Export : df_merged_clean.csv


In [8]:
# On repart du merged clean
df = df_merged.copy()
# On limite 2014-2024 (10 ans d'historique pour tout le monde)
df = df.dropna(subset=['code_commune_insee', 'annee', 'prixm2moyen', 'latitude', 'longitude'])
df = df[(df['annee'] >= 2014) & (df['annee'] <= 2024)].sort_values(['code_commune_insee', 'annee'])

# On ajoute 5 lags (2019-2023) pour prédire 2024 puis rolling
for lag in range(1, 6):
    df[f'prixm2moyen_lag{lag}'] = df.groupby('code_commune_insee')['prixm2moyen'].shift(lag)


In [9]:
from lightgbm import LGBMRegressor, early_stopping
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Features pour le modèle
features = ['annee', 'latitude', 'longitude'] + [f'prixm2moyen_lag{lag}' for lag in range(1, 6)]
df_train = df[df['annee'] <= 2023].dropna(subset=[f'prixm2moyen_lag{lag}' for lag in range(1, 6)] + ['latitude', 'longitude'])
X = df_train[features]
y = df_train['prixm2moyen']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.12, random_state=42)

model = LGBMRegressor(
    n_estimators=350,
    objective='regression',
    random_state=42,
    verbose=-1
)
model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='mae',
    callbacks=[early_stopping(100)]
)

# Affichage clair
best_iter = model.best_iteration_
best_mae = model.best_score_['valid_0']['l1']
best_mse = model.best_score_['valid_0']['l2']
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print("\n================ Résumé entraînement LightGBM ================")
print(f"👉 Early stopping : arrêt automatique après 100 itérations sans amélioration.")
print(f"👉 Meilleure itération atteinte : {best_iter}")
print(f"    - Erreur absolue moyenne (MAE, l1) sur validation : {best_mae:.2f} €")
print(f"    - Erreur quadratique moyenne (MSE, l2) sur validation : {best_mse:.2f}")
print("---------------------------------------------------------------")
print(f"MAE  global : {mae:.2f} €")
print(f"RMSE global : {rmse:.2f} €")
print(f"R²   global : {r2:.3f}")
print("===============================================================\n")


Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[327]	valid_0's l1: 257.335	valid_0's l2: 164060

================ Résumé entraînement LightGBM ================
👉 Early stopping : arrêt automatique après 100 itérations sans amélioration.
👉 Meilleure itération atteinte : 327
    - Erreur absolue moyenne (MAE, l1) sur validation : 257.33 €
    - Erreur quadratique moyenne (MSE, l2) sur validation : 164060.03
---------------------------------------------------------------
MAE  global : 257.33 €
RMSE global : 405.04 €
R²   global : 0.810



In [10]:
# Pour chaque commune, on part des prix 2020-2024 pour prédire 2025, puis rolling
df_2024 = df[df['annee'] == 2024][['code_commune_insee', 'latitude', 'longitude', 'prixm2moyen']]
for lag in range(1, 6):
    df_2024[f'prixm2moyen_lag{lag}'] = df.groupby('code_commune_insee')['prixm2moyen'].shift(lag).loc[df['annee'] == 2024].values

df_pred = df_2024.dropna(subset=[f'prixm2moyen_lag{lag}' for lag in range(1, 6)] + ['latitude', 'longitude']).copy()
df_pred['prixm2moyen_2024'] = df_pred['prixm2moyen'].astype(float)

for an in range(2025, 2030):
    X_pred = pd.DataFrame({
        'annee': [an]*len(df_pred),
        'latitude': df_pred['latitude'],
        'longitude': df_pred['longitude'],
        'prixm2moyen_lag1': df_pred['prixm2moyen_lag1'],
        'prixm2moyen_lag2': df_pred['prixm2moyen_lag2'],
        'prixm2moyen_lag3': df_pred['prixm2moyen_lag3'],
        'prixm2moyen_lag4': df_pred['prixm2moyen_lag4'],
        'prixm2moyen_lag5': df_pred['prixm2moyen_lag5'],
    })
    df_pred[f'prixm2moyen_{an}_pred'] = model.predict(X_pred)
    # On décale les lags pour l’année suivante (rolling forecast)
    for lag in range(5, 1, -1):
        df_pred[f'prixm2moyen_lag{lag}'] = df_pred[f'prixm2moyen_lag{lag-1}']
    df_pred['prixm2moyen_lag1'] = df_pred[f'prixm2moyen_{an}_pred']


In [ ]:
import plotly.express as px
import json
import urllib.request

# Colonnes finales et variation
cols_out = (
    ['code_commune_insee', 'latitude', 'longitude', 'prixm2moyen_2024'] +
    [f'prixm2moyen_{an}_pred' for an in range(2025, 2030)]
)
df_pred = df_pred[cols_out]
df_pred['variation_%'] = 100 * (df_pred['prixm2moyen_2029_pred'] - df_pred['prixm2moyen_2024']) / df_pred['prixm2moyen_2024']

# Carte
geojson_url = "https://france-geojson.gregoiredavid.fr/repo/communes.geojson"
with urllib.request.urlopen(geojson_url) as response:
    communes_geojson = json.load(response)

df_pred['code_commune_insee'] = df_pred['code_commune_insee'].astype(str).str.zfill(5)
def prix_fmt(val): return f"€ {int(round(val))}" if pd.notna(val) else "NA"
custom_data = [
    df_pred['prixm2moyen_2024'].apply(prix_fmt),
    df_pred['prixm2moyen_2025_pred'].apply(prix_fmt),
    df_pred['prixm2moyen_2026_pred'].apply(prix_fmt),
    df_pred['prixm2moyen_2027_pred'].apply(prix_fmt),
    df_pred['prixm2moyen_2028_pred'].apply(prix_fmt),
    df_pred['prixm2moyen_2029_pred'].apply(prix_fmt),
    df_pred['variation_%'].round(1)
]
fig = px.choropleth_map(
    df_pred,
    geojson=communes_geojson,
    locations='code_commune_insee',
    featureidkey='properties.code',
    color='variation_%',
    color_continuous_scale="RdYlGn",
    range_color=(df_pred['variation_%'].min(), df_pred['variation_%'].max()),
    center={"lat": 46.6, "lon": 2.6},
    zoom=5,
    opacity=0.80,
    hover_name='code_commune_insee',
    hover_data=None,
    custom_data=custom_data,
    title="Variation % du prix au m² entre 2024 et 2029 (LightGBM, rolling trend historique)"
)
fig.update_layout(
    autosize=False,
    width=1200,
    height=900,
    margin={"r":0,"t":50,"l":0,"b":0},
    legend_title_text='Variation (%)',
    font=dict(size=16),
    title_x=0.5,
    updatemenus=[dict(type="buttons", showactive=False,
        buttons=[dict(label="Plein écran", method="relayout", args=[{"width":1800, "height":1000}])])]
)
fig.update_traces(marker_line_width=0)
fig.update_traces(
    hovertemplate=
        "<b>INSEE = %{location}</b><br><br>" +
        "Prix 2024 = %{customdata[0]}<br>" +
        "Prix 2025 = %{customdata[1]}<br>" +
        "Prix 2026 = %{customdata[2]}<br>" +
        "Prix 2027 = %{customdata[3]}<br>" +
        "Prix 2028 = %{customdata[4]}<br>" +
        "Prix 2029 = %{customdata[5]}<br>" +
        "Variation 2024→2029 = %{customdata[6]} %<br>" +
        "<extra></extra>"
)
fig.show()

# Export CSV prévisions
df_pred.to_csv('predictions_prix_2025_2029_commune_lgbm.csv', index=False, encoding='utf-8')
print("Export : predictions_prix_2025_2029_commune_lgbm.csv")


In [13]:
# can you help me to bring the following columns: 'prixm2moyen_2024', 'prixm2moyen_2025_pred', 'prixm2moyen_2026_pred', 'prixm2moyen_2027_pred', 'prixm2moyen_2028_pred', 'prixm2moyen_2029_pred' into 1 column and add the year in a separate column?
df_long = pd.melt(df_pred,
    id_vars=['code_commune_insee', 'latitude', 'longitude'],
    value_vars=['prixm2moyen_2024', 'prixm2moyen_2025_pred', 'prixm2moyen_2026_pred', 'prixm2moyen_2027_pred', 'prixm2moyen_2028_pred', 'prixm2moyen_2029_pred'],
    var_name='annee',
    value_name='prixm2moyen'
)
df_long['annee'] = df_long['annee'].str.extract('.*(\d{4})').astype(int)
df_long = df_long.sort_values(['code_commune_insee', 'annee']).reset_index(drop=True)


<>:8: SyntaxWarning:

invalid escape sequence '\d'

<>:8: SyntaxWarning:

invalid escape sequence '\d'

/tmp/ipykernel_25027/3870567330.py:8: SyntaxWarning:

invalid escape sequence '\d'



In [14]:
# add code department
df_long['code_departement'] = df_long['code_commune_insee'].str[:2]
df_long['code_departement'] = df_long['code_departement'].astype(str).str.zfill(2)


In [15]:
df_long.drop(columns=['latitude', 'longitude'], inplace=True)


In [16]:
df_long


,code_commune_insee,annee,prixm2moyen,code_departement
0,01001,2024,3258.000000,01
1,01001,2025,2619.327443,01
2,01001,2026,2668.324830,01
3,01001,2027,2947.779988,01
4,01001,2028,3111.058689,01
...,...,...,...,...
173185,95690,2025,3264.446112,95
173186,95690,2026,3231.174089,95
173187,95690,2027,3415.801396,95
173188,95690,2028,3342.954621,95


In [17]:
df_long[df_long['code_commune_insee'].str.startswith("75")]


,code_commune_insee,annee,prixm2moyen,code_departement
142962,75056,2024,9674.000000,75
142963,75056,2025,8200.759621,75
142964,75056,2026,8200.759621,75
142965,75056,2027,8200.759621,75
142966,75056,2028,8200.759621,75
142967,75056,2029,8200.759621,75


In [18]:
dataset_communes = df_long.copy()


In [19]:
ROOT_DIR = "../data/"

def load_file_s3(object_key: str) -> pd.DataFrame:
    """Load a file from S3 and print its contents."""
    if not AWS_S3_BUCKET or not AWS_ACCESS_KEY_ID or not AWS_SECRET_ACCESS_KEY:
        raise ValueError(
            "AWS credentials or bucket name not set in environment variables."
        )

    s3_client = boto3.client(
        "s3",
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    )

    # check if the object already exists locally don't download it again
    if os.path.exists(f"{ROOT_DIR}{object_key}"):
        print(
            f"File {f'{ROOT_DIR}{object_key}'} already exists locally. Loading from local file."
        )
        return pd.read_csv(f"{ROOT_DIR}{object_key}")
    print(f"Downloading {f'{ROOT_DIR}{object_key}'} from S3 bucket {AWS_S3_BUCKET}.")

    response = s3_client.get_object(Bucket=AWS_S3_BUCKET, Key=object_key)
    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

    if status == 200:
        print(f"Successful S3 get_object response. Status - {status}")
        # save the file locally and create the directory if it doesn't exist
        os.makedirs(os.path.dirname(f"{ROOT_DIR}{object_key}"), exist_ok=True)
        # write the file to the local filesystem
        print(f"Saving {ROOT_DIR}{object_key} locally.")
        with open(f"{ROOT_DIR}{object_key}", "wb") as f:
            f.write(response["Body"].read())
        print(f"File {ROOT_DIR}{object_key} downloaded and saved locally.")
        # read the file into a DataFrame
        return pd.read_csv(f"{ROOT_DIR}{object_key}")
    raise ValueError(f"Unsuccessful S3 get_object response. Status - {status}")


In [20]:
# Get all unique communes from your existing data
dataset_insee_cities = load_file_s3("processed/referentiel/ref_espace_communes.csv")
dataset_insee_cities.drop(
    columns=["Unnamed: 0", "nom_commune_complet", "code_region", "nom_region"],
    inplace=True,
)
dataset_insee_cities.rename(
    columns={
        "code_commune_INSEE": "code_commune_insee",
    },
    inplace=True,
)
dataset_insee_cities["code_postal"] = (
    dataset_insee_cities["code_postal"].astype(str).str.zfill(5)
)
dataset_insee_cities["code_departement"] = (
    dataset_insee_cities["code_departement"].astype(str).str.zfill(2)
)

dataset_insee_cities.head()

all_communes = dataset_insee_cities["code_commune_insee"].unique()

min_year = 2024
max_year = 2029
all_years = range(min_year, max_year + 1)

all_combinations = list(itertools.product(all_communes, all_years))

dataset_communes_full = pd.DataFrame(
    all_combinations, columns=["code_commune_insee", "annee"]
)
# Filter out Paris communes as we don't have price data for them
dataset_communes_full = dataset_communes_full[
    ~dataset_communes_full["code_commune_insee"].isin(
        [
            "75101",
            "75102",
            "75103",
            "75104",
            "75105",
            "75106",
            "75107",
            "75108",
            "75109",
            "75110",
            "75111",
            "75112",
            "75113",
            "75114",
            "75115",
            "75116",
            "75117",
            "75118",
            "75119",
            "75120",
        ]
    )
]

print("\nTemplate DataFrame with all combinations:")
display(dataset_communes_full.head())
print(f"Total combinations: {len(dataset_communes_full)}")

all_departments = dataset_insee_cities["code_departement"].unique()

all_combinations = list(itertools.product(all_departments, all_years))
dataset_departement_full = pd.DataFrame(
    all_combinations, columns=["code_departement", "annee"]
)

print("\nTemplate DataFrame with all combinations:")
display(dataset_departement_full.head())
print(f"Total combinations: {len(dataset_departement_full)}")


File ../data/processed/referentiel/ref_espace_communes.csv already exists locally. Loading from local file.

Template DataFrame with all combinations:


,code_commune_insee,annee
0,01001,2024
1,01001,2025
2,01001,2026
3,01001,2027
4,01001,2028


Total combinations: 214644

Template DataFrame with all combinations:


,code_departement,annee
0,01,2024
1,01,2025
2,01,2026
3,01,2027
4,01,2028


Total combinations: 576


In [21]:
dataset_departements = (
    df_long.groupby(["code_departement", "annee"])["prixm2moyen"].mean().reset_index()
)
dataset_departements.head()


,code_departement,annee,prixm2moyen
0,01,2024,2442.246684
1,01,2025,2609.276530
2,01,2026,2748.942376
3,01,2027,2892.377294
4,01,2028,3033.574608


In [22]:
MISSING_VALUE_PLACEHOLDER = -1

# Departement dataset
missing_rows = dataset_departement_full[
    ~dataset_departement_full.set_index(
        ["code_departement", "annee"]
    ).index.isin(
        dataset_departements.set_index(
            ["code_departement", "annee"]
        ).index
    )
]
missing_rows = missing_rows[["code_departement", "annee"]]
missing_rows["prixm2moyen"] = (
    MISSING_VALUE_PLACEHOLDER  # Set a default value for prixm2moyen
)
dataset_departements = pd.concat(
    [dataset_departements, missing_rows], ignore_index=True
)

missing_rows = dataset_communes_full[
    ~dataset_communes_full.set_index(["code_commune_insee", "annee"]).index.isin(
        dataset_communes.set_index(["code_commune_insee", "annee"]).index
    )
]
missing_rows = missing_rows[["code_commune_insee", "annee"]]
missing_rows["prixm2moyen"] = (
    MISSING_VALUE_PLACEHOLDER  # Set a default value for prixm2moyen
)
# print(f"Dataset shape before adding missing rows: {dataset_communes.shape}")
dataset_communes = pd.concat(
    [dataset_communes, missing_rows], ignore_index=True
)
dataset_communes["code_departement"] = dataset_communes[
    "code_commune_insee"
].str[:2]
# print("\nDataset with all combinations including missing rows:")
# display(dataset_communes.head())
# print(f"Dataset shape after adding missing rows: {dataset_communes.shape}")


In [23]:
s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
)
dataset_housing_prices_predictions_key = (
    "processed/predictions/dataset_communes_prices.csv"
)
s3_client.put_object(
    Bucket=AWS_S3_BUCKET,
    Key=dataset_housing_prices_predictions_key,
    Body=dataset_communes.to_csv(index=False, encoding="utf-8"),
)

s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
)
dataset_housing_prices_predictions_key = (
    "processed/predictions/dataset_departements_prices.csv"
)
s3_client.put_object(
    Bucket=AWS_S3_BUCKET,
    Key=dataset_housing_prices_predictions_key,
    Body=dataset_departements.to_csv(index=False, encoding="utf-8"),
)


{'ResponseMetadata': {'RequestId': 'BM85MQ7CGB3SGPM5',
  'HostId': '/Fh5tQT5wWpeNGM/QORSyjGydxelTQfTlJQRDZ3m9l10Yg1pOtTDEJWG8kP6KbkVbV8fxTGY7tijhRnfTs5DpGMI4LqS1wXo',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '/Fh5tQT5wWpeNGM/QORSyjGydxelTQfTlJQRDZ3m9l10Yg1pOtTDEJWG8kP6KbkVbV8fxTGY7tijhRnfTs5DpGMI4LqS1wXo',
   'x-amz-request-id': 'BM85MQ7CGB3SGPM5',
   'date': 'Wed, 30 Jul 2025 07:50:13 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"0ecbf37648c665710119a84131d6a3a9"',
   'x-amz-checksum-crc32': 'Q/cgRg==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 1},
 'ETag': '"0ecbf37648c665710119a84131d6a3a9"',
 'ChecksumCRC32': 'Q/cgRg==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}